In [411]:
# Import Dependencies
import pandas as pd
import os as os
import numpy as np

In [412]:
# Make a reference to the purchase_data.json file path
path = "Heroes-of-Pymoli.json"
# Import the purchase_data.json file as a DataFrame
df_all_data = pd.read_json(path,orient = 'columns')
df_all_data.head(10)
    

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


In [413]:
# Remove unecessary columns from the DataFrame and save the new DataFrame
df_ordered = df_all_data[["SN", "Age", "Gender", "Item ID",
                       "Item Name", "Price"]]
df_ordered.reset_index()
gender_df = df_ordered.loc[:, ["Gender", "SN", "Age"]].drop_duplicates()


In [414]:
# Count of Players

count_players = len(df_ordered["SN"].unique())
count_players_df = pd.DataFrame(dict([ ["Count of Players", [count_players]]
                                     ]))
count_players_df
new_df = pd.DataFrame({"Total Players": [count_players]})
new_df

,Total Players
0,573


In [415]:
# Purchasing Analysis (Total)
    # Number of Unique Items
    # Average Purchase Price
    # Total Number of Purchases
    # Total Revenue

unique_items = len(df_ordered["Item ID"].unique())
average_purchase_price = '$'+str(round(sum(df_ordered["Price"])/len(df_ordered["Price"]),2))
number_of_purchases = len(df_ordered["Item ID"])
total_revenue = '$'+ str(round(sum(df_ordered["Price"]),2))

purchase_analysis_df = pd.DataFrame(dict([ ['Number of Unique Items', [unique_items]], 
                                           ['Average Price', [average_purchase_price]],
                                           ['Number of Purchases', [number_of_purchases]],
                                           ['Total Revenue', [total_revenue]]
                                         ]))

purchase_analysis_df

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.93,780,183,$2286.33


In [416]:
# Gender Demographics

    # Percentage and Count of Male Players
    # Percentage and Count of Female Players
    # Percentage and Count of Other / Non-Disclosed   
    
total_player_count = len(df_ordered["SN"])
unique_player_count = len(df_ordered["SN"].unique())

count_df = gender_df["Gender"].value_counts()
percent_df = (count_df/count_players)*100
others_df = total_player_count - (male_count + female_count)
gender_demographics_df = pd.DataFrame({"Total Count": count_df, "Percentages of Players": percent_df})

gender_demographics_df

# Push the remade DataFrame to a new CSV file
#rename_df.to_csv("Output/BooksNewData.csv")

,Percentages of Players,Total Count
Male,81.151832,465
Female,17.452007,100
Other / Non-Disclosed,1.396161,8


In [417]:
# Purchasing Analysis (Gender) 

    # The below each broken by gender

        # Purchase Count
        # Average Purchase Price
        # Total Purchase Value
        # Normalized Totals 

purchase_df = df_ordered.loc[:, ["Gender", "Price", "Age", "SN", "Item Name", "Item ID"]]

gender_count = purchase_df["Gender"].value_counts()

sum_price = purchase_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")


avg_purchse_price = round(sum_price/gender_count,2)

total_purchase_value = round(avg_purchse_price * gender_count,2)

normalized_total = round(total_purchase_value/gender_demographics_df['Total Count'],2)

purchase_analysis_df = pd.DataFrame({"Purchase Count": gender_count, "Average Purchase Price": avg_purchse_price,
                                    "Total Purchase Value": total_purchase_value, "Normalized Totals": normalized_total})
purchase_analysis_df


,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
Female,2.82,3.84,136,383.52
Male,2.95,4.02,633,1867.35
Other / Non-Disclosed,3.25,4.47,11,35.75


In [418]:
# Age Demographics

    # The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.) 

        # Purchase Count
        # Average Purchase Price
        # Total Purchase Value
        # Normalized Totals

#age_df = df_ordered.loc[:, ["Gender", "Price", "Age", "SN"]] 

age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=labels)

age_demographics_totals = player_demographics["Age Ranges"].value_counts()
age_demographics_totals
age_demographics_percents = age_demographics_totals / num_players * 100
age_demographics = pd.DataFrame({"Total Count": age_demographics_totals, "Percentage of Players": age_demographics_percents})


age_demographics = age_demographics.round(2)

age_demographics.sort_index()

,Percentage of Players,Total Count
<10,3.32,19
10-14,4.01,23
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-39,4.71,27
40+,1.92,11


In [419]:
        # Purchase Count
        # Average Purchase Price
        # Total Purchase Value
        # Normalized Totals

# Bin
purchase_df["Age Ranges"] = pd.cut(purchase_df["Age"], age_bins, labels=labels)    

age_purchase_total = purchase_df.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_average = purchase_df.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_counts = purchase_df.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")

normalized_total = age_purchase_total / age_demographics["Total Count"]


age_df = pd.DataFrame({"Purchase Count": age_counts, "Average Purchase Price": age_average, "Total Purchase Value": age_purchase_total, "Normalized Totals": normalized_total})

age_df

,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
10-14,2.770000,4.215217,35,96.95
15-19,2.905414,3.864200,133,386.42
20-24,2.913006,3.779035,336,978.77
25-29,2.962640,4.256667,125,370.33
30-34,3.082031,4.196809,64,197.25
35-39,2.842857,4.422222,42,119.40
40+,3.161765,4.886364,17,53.75
<10,2.980714,4.392632,28,83.46


In [420]:
# Top Spenders

    # Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

        # SN
        # Purchase Count
        # Average Purchase Price
        # Total Purchase Value


spender_total = purchase_df.groupby(["SN"]).sum()["Price"]
spender_average = purchase_df.groupby(["SN"]).mean()["Price"]
spender_count = purchase_df.groupby(["SN"]).count()["Price"]


spender_data = pd.DataFrame({"Total Purchase Value": spender_total, "Average Purchase Price": spender_average, "Purchase Count": spender_count})


spender_sorted = spender_data.sort_values("Total Purchase Value", ascending=False)




spender_sorted.head()

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Undirrala66,3.412000,5,17.06
Saedue76,3.390000,4,13.56
Mindimnya67,3.185000,4,12.74
Haellysu29,4.243333,3,12.73
Eoda93,3.860000,3,11.58


In [421]:
# Most Popular Items

    # Identify the 5 most popular items by purchase count, then list (in a table):

        # Item ID
        # Item Name
        # Purchase Count
        # Item Price
        # Total Purchase Value


item_df = purchase_df.loc[:,["Item ID", "Item Name", "Price"]]

total_purchase = item_df.groupby(["Item ID", "Item Name"]).sum()["Price"]
average_purchase = item_df.groupby(["Item ID", "Item Name"]).mean()["Price"]
item_count = item_df.groupby(["Item ID", "Item Name"]).count()["Price"]


item_df = pd.DataFrame({"Total Purchase Value": total_purchase, "Item Price": average_purchase, "Purchase Count": item_count})

item_sorted = item_df.sort_values("Purchase Count", ascending=False)

item_pop = item_sorted.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

item_pop.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,2.35,25.85
84,Arcane Gem,11,2.23,24.53
31,Trickster,9,2.07,18.63
175,Woeful Adamantite Claymore,9,1.24,11.16
13,Serenity,9,1.49,13.41


In [423]:
# Most Profitable Items

    # Identify the 5 most profitable items by total purchase value, then list (in a table):

        # Item ID
        # Item Name
        # Purchase Count
        # Item Price
        # Total Purchase Value


purchase_item = item_df.sort_values("Total Purchase Value", ascending=False)

profitable_item = purchase_item.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

profitable_item.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,4.14,37.26
115,Spectral Diamond Doomblade,7,4.25,29.75
32,Orenmir,6,4.95,29.70
103,Singed Scalpel,6,4.87,29.22
107,"Splitter, Foe Of Subtlety",8,3.61,28.88
